# PHISIONET 2025 model training notebook

## Imports

In [120]:
import pickle as pkl 
import pandas as pd 
import os 
import importlib
import utils
importlib.reload(utils) 
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix




## Data Loading

In [54]:
training_set, labels = utils.load_training_set("C:/Users/emanu/OneDrive/Desktop/Physione2025Data/Training_data/2025-03-17 101119")

3268 observations, cropped to the smalles, no prep


torch.Size([3268, 1568, 12])

(3268,)

(3268,)

## Trainig Test split

In [58]:
training_set_np = training_set.numpy()

In [59]:
X_train, X_test, y_train, y_test = train_test_split(training_set_np,labels,test_size=0.2,random_state=42)

In [130]:
X_train_tensor.shape
X_test_tensor.shape

torch.Size([654, 1568, 12])

In [111]:
y_train

array([0., 1., 0., ..., 0., 0., 0.])

## Model Definition 


In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor.to(device)
y_train_tensor.to(device)
model = utils.TimeSeriesCNN(2).to(device)

In [147]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output.squeeze(),y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [121]:
model.eval()

TimeSeriesCNN(
  (conv1): Conv1d(12, 32, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=24960, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [131]:
with torch.no_grad():
    output_test = model(X_test_tensor)
    predictions = (output_test > 0.5).float()

In [132]:
predictions.shape

torch.Size([654, 1])

In [133]:
y_test_tensor.shape

torch.Size([654])

In [134]:
y_test_np = y_test_tensor.cpu().numpy()  # Converti a numpy
predictions_np = predictions.cpu().numpy()  # Converti a numpy

accuracy = accuracy_score(y_test_np, predictions_np)
precision = precision_score(y_test_np, predictions_np)
recall = recall_score(y_test_np, predictions_np)
f1 = f1_score(y_test_np, predictions_np)
conf_matrix = confusion_matrix(y_test_np, predictions_np)

# Stampa i risultati
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Confusion Matrix:\n', conf_matrix)

Accuracy: 0.6315
Precision: 0.6544
Recall: 0.6257
F1 Score: 0.6398
Confusion Matrix:
 [[199 113]
 [128 214]]
